# 实验二

## 逻辑回归与批量随机梯度下降

### 处理数据

In [1]:
import sklearn,numpy,jupyter,matplotlib
import os
import sklearn.datasets
import numpy as np
import random

In [2]:
X_train, Y_train = sklearn.datasets.load_svmlight_file('a9a.txt')
X_test, Y_test = sklearn.datasets.load_svmlight_file('a9a.t.txt', n_features=123)

X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((32561, 123), (32561,), (16281, 123), (16281,))

In [3]:
X_train = X_train.toarray()
X_test = X_test.toarray()
Y_train = Y_train.reshape(len(Y_train),1)
Y_test = Y_test.reshape(len(Y_test),1)
X_train = np.concatenate((np.ones((X_train.shape[0],1)), X_train), axis = 1)
X_test = np.concatenate((np.ones((X_test.shape[0], 1)), X_test), axis = 1)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

((32561, 124), (32561, 1), (16281, 124), (16281, 1))

In [4]:
X_train, Y_train, X_test, Y_test

(array([[1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]),
 array([[-1.],
        [-1.],
        [-1.],
        ...,
        [-1.],
        [-1.],
        [ 1.]]),
 array([[1., 1., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 1., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]),
 array([[-1.],
        [-1.],
        [ 1.],
        ...,
        [-1.],
        [-1.],
        [ 1.]]))

### 构建函数

In [5]:
def compute_MSELoss(X, Y, W):
    return np.power((X.dot(W) - Y), 2).mean()/2

In [6]:
def compute_gardient(X, Y, W):
    return -X.T.dot(Y - X.dot(W))

In [82]:
def compute_accuracy(X, Y, W, threshold):
    result = X.dot(W)
    TPandFN = 0
    for (i, y) in zip(result, Y):
        if((i - threshold)*y > 0):
            TPandFN += 1
            # print(i, threshold, y)
    return TPandFN/(X.shape[0])

In [129]:
def thirdnarySearch(arr, l, r, X, Y, W):
    r -= 1
    while l+1< r:
        mid = (l + r) // 2
        midr = (mid + r) // 2
        
        accuracyMid = compute_accuracy(X, Y, W, arr[mid])
        accuracyR = compute_accuracy(X, Y, W, arr[r])

        # print(l, mid, r, accuracyL, accuracyMid, accuracyR)
        # if(accuracyL==accuracyR and accuracyL==accuracyMid):
        #     break

        if accuracyR >= accuracyMid:
            l = mid
        else:
            r = midr
        print(l, mid, midr, r, accuracyMid)
    return arr[l]

In [51]:
def train(X, Y, batch_size=1, epoch=30, learning_rate=0.05, optimizer='SGD'):
    random.seed(8848)
    batch = random.sample(list(np.c_[X, Y]), batch_size)
    batch = np.array(batch)
    X_batch = batch[:, :-1]
    Y_batch = batch[:, -1:]
    W = np.random.rand(X_batch.shape[1], 1)
    # print(W.shape)
    #进行优化
    if optimizer == 'SGD':
        for i in range(int(epoch)):
            W -= learning_rate*compute_gardient(X_batch, Y_batch, W)
    #计算最优阈值
    threshold = thirdnarySearch(sorted(X.dot(W)), 0, len(sorted(X.dot(W))), X, Y, W)
    
    return W, threshold

### 基准线和拟合

In [9]:
# baseline
W = np.random.rand(X_train.shape[1], 1)
compute_MSELoss(X_train, Y_train, W), compute_MSELoss(X_test, Y_test, W)

(32.905513220143426, 32.90046445472171)

SGD算法

In [130]:
batch_size=4
# epoch=300
learning_rate=0.05
optimizer='SGD'
threshold = []
W = []
for i in range(4):
    tempA, tempB = train(X_train, Y_train, batch_size=1, epoch=np.power(10, i), learning_rate=0.05, optimizer='SGD')
    W.append(tempA)
    threshold.append(tempB)
    print(i)

16280 16280 24420 32560 0.640551580111176
24420 24420 28490 32560 0.7452473818371672
24420 28490 30525 30525 0.7713215196093486
27472 27472 28998 30525 0.7673904364116582
27472 28998 29761 29761 0.7742698320076165
28616 28616 29188 29761 0.7718743281840238
28616 29188 29474 29474 0.7741162740702067
28616 29045 29259 29259 0.7743312551825804
28616 28937 29098 29098 0.7739012929578329
28857 28857 28977 29098 0.7736248886704954
28857 28977 29037 29037 0.7743005435950985
28947 28947 28992 29037 0.7740855624827248
28947 28992 29014 29014 0.7742698320076165
28947 28980 28997 28997 0.7743312551825804
28972 28972 28984 28997 0.7741776972451706
28984 28984 28990 28997 0.7742391204201345
28990 28990 28993 28997 0.7741776972451706
28990 28993 28995 28995 0.7742698320076165
28992 28992 28993 28995 0.7742698320076165
28993 28993 28994 28995 0.7742698320076165
28993 28994 28994 28994 0.7743005435950985
0
16280 16280 24420 32560 0.5558490218359387
24420 24420 28490 32560 0.6638309634224993
28490 2849

In [131]:
len(W), W[0].shape, threshold

(4,
 (124, 1),
 [array([6.51184824]),
  array([7.41085678]),
  array([7.70052198]),
  array([6.24880457])])

In [136]:
compute_accuracy(X_train, Y_train, W[0], threshold[0][0]), threshold

(0.7742698320076165,
 [array([6.51184824]),
  array([7.41085678]),
  array([7.70052198]),
  array([6.24880457])])